# Big Data Project

## Configuration

In [1]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [102]:
//Denys: val bucketname = "unibo-bd2122-dgrushchak"
//Riccardo: val bucketname = "unibo-bd2223-rbacca"

val bucketname ="unibo-bd2122-dgrushchak"

//Denys: val path_flights_db = "s3a://"+bucketname+"/projectNoDuplication/part-00000"
//Denys: val path_flights_db = "s3a://"+bucketname+"/projectTest/NoDuplication/part-00000"
//Riccardo:val path_flights_db = "s3a://"+bucketname+"/bigdata-project/part_1.txt"

val path_flights_db =  "s3a://"+bucketname+"/project/small_file_10000000.txt"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2122-dgrushchak
path_flights_db: String = s3a://unibo-bd2122-dgrushchak/project/small_file_10000000.txt
res228: String = application_1675070277172_0002
res230: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1675070277172_0002/


### Parsing
This parser make flights flat on base of number of the leg.

If a flight has two legs it will be splitted into 2 rows that:
* duplicate for each row the fields as: id, flightDate, startingAirport, destinationAirport, isNonStop, isBasicEconomy.
* has one value of the two that was saved together in fields as: airplaneType, airlineName, segmentDistance, segmentDuration.
* recalculate a totalFare for each row on base of segmentDistance. Sum of two row must return the original value

id is a new field that replace the orinal identifier

In [103]:
type MyTuple = (String, String, String, String, String, Boolean, Boolean, Double, Int, Int)
object FlatParser{
    import scala.collection.mutable.ArrayBuffer
    import scala.collection.mutable.ListBuffer
    val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val doubleVerticalLine = "\\|\\|"
    
    case class FlatFlightData(
        id: Long,                   //1
        flightDate: String,         //2      
        startingAirport: String,    //3 
        destinationAirport: String, //4
        airplaneType: String,       //5
        airlineName: String,        //6
        isNonStop: Boolean,         //7 
        isBasicEconomy: Boolean,    //8 
        totalFare: Double,          //9
        segmentDuration: Int,     //10
        segmentDistance: Int      //11
    ){
        // return a tuple of the object
        def un() = FlatFlightData.unapply(this).get 
    }
    
    def convertAirplaneType(value: String): Array[String] = {
        //Ex. input Airbus A320||Boeing 737-800
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result
    }
    
    def convertAirlineName(value: String): Array[String] = {
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result
    }
    
    //return array of duration of each leg in minutes
    def convertSegmentsDuration(value: String): Array[Int] = {
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result.map(_.toInt/60)
    }
    
    def convertSegmentsDistance(value: String): Array[Int] = {
        val result = value.split(doubleVerticalLine)
        if(result.head == "" || result.head == "None") Array.empty
        else result.map(_.toInt)
    }
   
    def parseFlightInformationLine(line: String): Option[ListBuffer[MyTuple]] = {
        try {
            val input = line.split(commaRegex)
            
            val airplanes = convertAirplaneType(input(23)) //airplane type [String||String] 
            val airline = convertAirlineName(input(21)) // airliine name  [String||String]
            val segmentsDuration = convertSegmentsDuration(input(24)) // segments duration in seconds (but the function return duration in minutes)
            val segmentsDistance = convertSegmentsDistance(input(25))// segment distance in milles: Vector of [Int||Int]  can be [None||None]
            if (airplanes.isEmpty || 
                airline.isEmpty || 
                segmentsDuration.isEmpty || 
                segmentsDistance.isEmpty|| 
                airplanes.length != airline.length ||
                airplanes.length != segmentsDuration.length ||
                airplanes.length != segmentsDistance.length ) None
            else {
                val arr: ListBuffer[MyTuple] = ListBuffer()
                val distanceSum = segmentsDistance.reduce(_+_).toDouble
                for (i <- 0 until airplanes.length){
                    val legPriceBasedOnDistance: Double = (segmentsDistance(i).toDouble/distanceSum) * input(12).toDouble //12 baseFare in $
                    arr += ((input(2), //flightdate
                             input(3), //starting airport
                             input(4), //destination airport
                             airplanes(i),
                             airline(i),
                             input(10).toBoolean,  //isNonStop
                             input(8).toBoolean,  //isBasicEconomy
                             legPriceBasedOnDistance,
                             segmentsDuration(i),
                             segmentsDistance(i))
                    )
                }
                Some(arr)
            }
        } catch {
            case _: Exception => None
        }
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined type alias MyTuple
defined object FlatParser


### Create and save dataset without duplication
The original dataset has duplicated record and their removing can take very long time.
We decide to create a new dataset that has original format but have not duplicated records
> This code must be executed only once then produced files can be used as sources

This code:
1. Read and parse the original data
2. Create a new numeric identifier
3. Replace original textual identifier and insert the data in FlatFlighData class
4. Create a cache

In [108]:
sc.getPersistentRDDs.foreach(_._2.unpersist()) 
import org.apache.spark.storage.StorageLevel._
//zipWithIndex trigger a spark job, for not parse twice all the data we save parsed data on the disk
val parsedData = sc.textFile(path_flights_db).flatMap(FlatParser.parseFlightInformationLine).persist(DISK_ONLY)

val rddFlightsCached = parsedData.
    zipWithIndex(). // Introduce numeric id for each flight row
    map({case (v, i) => (i, v)}).
    flatMapValues(m => m).
    map({case (k, v) => FlatParser.FlatFlightData.apply(k, v._1, v._2, v._3, v._4, v._5, v._6, v._7, v._8, v._9, v._10)}).cache
//     map({case (k, v) => FlatParser.FlatFlightData.apply(k, v._2, v._3, v._4, v._5, v._6, v._7, v._8, v._9, v._10, v._11)}).cache

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.storage.StorageLevel._
parsedData: org.apache.spark.rdd.RDD[scala.collection.mutable.ListBuffer[MyTuple]] = MapPartitionsRDD[368] at flatMap at <console>:29
rddFlightsCached: org.apache.spark.rdd.RDD[FlatParser.FlatFlightData] = MapPartitionsRDD[372] at map at <console>:32


In [110]:
rddFlightsCached.map(_.un()).take(10).foreach(println)
// rdd.map(_.un()).filter(f => f._3 == "ATL" && f._4 == "BOS" && f._5 == "Airbus A320" && f._6 == "American Airlines").sortBy(s => s._2).take(10).foreach(println)

def round(v: Double): Double = {
    (v*100).toInt/100.toDouble
}

val numberOfRecords = rddFlightsCached.map(_.id).distinct.count
val totalNumberOfTuples = rddFlightsCached.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0,2022-04-17,ATL,BOS,Airbus A321,Delta,true,false,248.6,149,947)
(1,2022-04-17,ATL,BOS,Airbus A321,Delta,true,false,248.6,150,947)
(2,2022-04-17,ATL,BOS,Boeing 757-200,Delta,true,false,248.6,150,947)
(3,2022-04-17,ATL,BOS,Airbus A321,Delta,true,false,248.6,152,947)
(4,2022-04-17,ATL,BOS,Airbus A321,Delta,true,false,248.6,154,947)
(5,2022-04-17,ATL,BOS,Airbus A320,American Airlines,false,false,59.885774058577404,90,228)
(5,2022-04-17,ATL,BOS,Airbus A320,American Airlines,false,false,191.2142259414226,125,728)
(6,2022-04-17,ATL,BOS,Airbus A320,American Airlines,false,false,59.885774058577404,90,228)
(6,2022-04-17,ATL,BOS,Boeing 737-800,American Airlines,false,false,191.2142259414226,138,728)
(7,2022-04-17,ATL,BOS,Airbus A319,American Airlines,false,false,59.885774058577404,81,228)
round: (v: Double)Double
numberOfRecords: Long = 8636358
totalNumberOfTuples: Long = 15315726


## Explorative queries

1. How many distinct airports and aircraft models
2. Average travel duration for airline, only in non-stop flights
3. Percentage of basic economy tickets, based on all tickets
4. Percentage of non-stop flights (flights with one leg)
5. Average and price range of tickets
6. Average ticket price for each airline
7. Average and range of travel distance
8. Top 10 airports with more arriving flights


In [94]:
//1.How many distinct airports and aircraft models
val distinctAirports = rddFlightsCached.
    map(x => x.startingAirport).
    distinct.
    union(rddFlightsCached.map(x => x.destinationAirport).distinct).
    distinct.count
val distinctAircraftModels = rddFlightsCached.flatMap(x => x.airplaneType).distinct.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

distinctAirports: Long = 16
distinctAircraftModels: Long = 53


In [95]:
//2. Average travel duration for each airline, in non stop flights
val distinctAirlines = rddFlightsCached.
    map(x => (x.airlineName, x.segmentDuration)).
    aggregateByKey((0.0, 0.0))((a,v)=>(a._1+v, a._2+1), (a1,a2)=>(a1._1+a2._1, a1._2+a2._2)).
    map({case(k,v)=>(k,v._1/v._2)}).
    collect().
    foreach({case (airline, value) => println(airline + " => " + round(value) + " avg minutes")})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Frontier Airlines => 154.08 avg minutes
Sun Country Airlines => 178.57 avg minutes
United => 161.23 avg minutes
Southern Airways Express => 64.43 avg minutes
Cape Air => 68.81 avg minutes
Contour Airlines => 82.26 avg minutes
Delta => 140.5 avg minutes
Key Lime Air => 97.99 avg minutes
Boutique Air => 85.36 avg minutes
Alaska Airlines => 197.43 avg minutes
American Airlines => 147.45 avg minutes
JetBlue Airways => 189.25 avg minutes
distinctAirlines: Unit = ()


In [96]:
//3. Percentage of basic economy tickets, based on all tickets
round((rddFlightsCached.filter(_.isBasicEconomy).map(_.id).distinct.count.toDouble/numberOfRecords).toDouble*100) + " %"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res208: String = 8.73 %


In [97]:
//4. Percentage of non-stop flights (flights with one leg)
round((rddFlightsCached.filter(_.isNonStop).map(_.id).distinct.count.toDouble/numberOfRecords).toDouble*100) + " %"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res210: String = 29.7 %


In [98]:
//5. Average and price range of tickets
val ticketPrices = rddFlightsCached.map(x => (x.id,x.totalFare)).reduceByKey(_+_).map(_._2)
"Range of prices: "  + ticketPrices.min + " to " + ticketPrices.max
"Avg price: " + round((ticketPrices.sum/numberOfRecords).toDouble)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ticketPrices: org.apache.spark.rdd.RDD[Double] = MapPartitionsRDD[335] at map at <console>:24
res212: String = Range of prices: 23.97 to 3810.6
res213: String = Avg price: 356.33


In [99]:
//6. Average ticket price for each airline
val avgTicketPricePerAirline = rddFlightsCached.
    map(x => (x.airlineName,(x.totalFare, 1))).
    reduceByKey((a,b) => (a._1+b._1, a._2+b._2)).
    map(m => (m._1,m._2._1/m._2._2)).
    collect.foreach({case (name, value) => println(name + " => " + round(value))})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Frontier Airlines => 139.53
Sun Country Airlines => 203.29
United => 214.76
Southern Airways Express => 69.29
Cape Air => 94.25
Contour Airlines => 133.7
Delta => 188.15
Key Lime Air => 126.02
Boutique Air => 143.0
Alaska Airlines => 326.07
American Airlines => 170.37
JetBlue Airways => 194.28
avgTicketPricePerAirline: Unit = ()


In [100]:
//7. Average and range of travel distance
val travelDistances = rddFlightsCached.map(x => (x.id, x.segmentDistance)).reduceByKey(_+_).map(_._2)
"Range of distances: "  + travelDistances.min + " to " + travelDistances.max
"Avg travel distance: " + round((travelDistances.sum/numberOfRecords).toDouble)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

travelDistances: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[341] at map at <console>:24
res216: String = Range of distances: 97 to 4414
res217: String = Avg travel distance: 1564.28


In [101]:
//8. Top 10 airports with more arriving flights
val topAirports = rddFlightsCached.
    map(x => (x.id, x.destinationAirport)).
    distinct().
    map(m => (m._2, 1)).
    reduceByKey(_+_).    
    sortBy(_._2, false).
    take(10).
    foreach({case (name, value) => println(name + " => " + value)})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LAX => 40093
LGA => 34474
MIA => 31056
BOS => 30398
DFW => 29718
ORD => 29543
CLT => 29287
SFO => 29191
ATL => 27818
DEN => 27670
topAirports: Unit = ()


## Main queries


In [ ]:
val path_output = "s3a://"+bucketname+"/spark/avgRatPerMovie" //todo

### Riccardo

> Riccardo: Aggrego su aeroporto di arrivo e di partenza, calcolando per ogni coppia un AVG dei prezzi dei biglietti. Join con dataset originale. Aggrego nuovamente sulla classificazione di prima e sulla distanza totale percorsa. Ottengo così per ogni tratta il prezzo medio e la distanza media percorsa.

In [ ]:
// Query approfondita --> Riccardo

// Prima aggregate poi Join
val result = rddFlightsCached.
    map(x => ((x.startingAirport, x.destinationAirport), (x.totalFare, 1))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v._1, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
    map({case (k,(sum,cnt)) => (k, (sum/cnt))}).
    join(rddFlightsCached.map(x => ((x.startingAirport, x.destinationAirport), x.segmentDistance)).
        aggregateByKey((0.0,0.0))((a,v)=>(a._1+v, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
        map({case (k,(sum,cnt)) => (k, (sum/cnt))})).
    sortByKey().
    take(10).foreach(println)


// Prima Join poi aggregate
/*val result = rddFlightsCached.
    map(x => ((x.startingAirport, x.destinationAirport), (x.totalFare, 1))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v._1, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
    map({case (k,(sum,cnt)) => (k, (sum/cnt))}).
    join(rddFlightsCached.map(x => ((x.startingAirport, x.destinationAirport), x.segmentDistance))). //([startingAirport, arrivingAirport], [prezzo medio, distanza percorsa])
    aggregateByKey((0.0,0.0,0.0))((a,v)=>(a._1+v._2, a._2+1, v._1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2,a1._2)).
    map({case (k,(sum,cnt,avg)) => (k, (sum/cnt,avg))}).
    map({case (k,(v1,v2)) => (k, (v2,v1))}).
    sortByKey().
    take(10).foreach(println)*/




### Denys
> Denys: aggrego su “aircraft model” per calcolare la “segment distance” totale percorsa da ogni modello, poi faccio self-join e aggregazione per determinare il “travel duration” per ogni “aircraft model”. Alla fine determino la velocità di ogni modello partendo dai dati aggregati.

In [ ]:
rddFlightsCached.map(x => (x.airplaneType, x.segmentsDistance)).zipWithIndex.take(10)
rddFlightsCached.map(x => (x.airplaneType, x.segmentsDuration)).zipWithIndex.take(10)